# Image Classifier using Convolutional Neural Networks

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Hyper Parameters
num_epochs = 400
batch_size = 64
learning_rate = 0.001

In [ ]:
#ImageNet Assumptions
normalize = transforms.Normalize(
	mean=[0.485, 0.456, 0.406],
	std=[0.229, 0.224, 0.225]
)

#Normalize -> Tensor -> Data augmentations (Random cropping and flipping)
preprocess = transforms.Compose([
	transforms.CenterCrop(100),
	transforms.RandomHorizontalFlip(),
	transforms.ToTensor(),
	normalize
])

In [ ]:
# Data Set
train_dataset = dsets.ImageFolder(root="train", transform=preprocess)

# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Data Set
test_dataset = dsets.ImageFolder(root="test", transform=transforms.ToTensor())

# Data Loader
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# CNN Model (2 conv layer)
class CNN(nn.Module):
	def __init__(self):
		super(CNN, self).__init__()
		self.layer1 = nn.Sequential(
			nn.Conv2d(3, 16, kernel_size=3, padding=1),
			nn.BatchNorm2d(16),
			nn.ReLU(),
			nn.MaxPool2d(2))
		self.layer2 = nn.Sequential(
			nn.Conv2d(16, 32, kernel_size=5, padding=2),
			nn.BatchNorm2d(32),
			nn.ReLU(),
			nn.MaxPool2d(2))
		self.fc1 = nn.Sequential(
			nn.Linear(20000, 500),
			nn.Dropout(0.75),
			nn.ReLU())
		self.fc2 = nn.Sequential(
			nn.Linear(500, 50),
			nn.Dropout(0.75),
			nn.ReLU())
		self.fc3 = nn.Sequential(
			nn.Linear(50, 5),
			nn.Dropout(0.75),
			nn.LogSoftmax())

	def forward(self, x):
		out = self.layer1(x)
		out = self.layer2(out)
		out = out.view(out.size(0), -1) #Flatten out the conv layer to fc layer
		out = self.fc1(out)
		out = self.fc2(out)
		out = self.fc3(out)
		return out

In [ ]:
cnn = CNN()
#cnn.load_state_dict(torch.load('cnn.pkl'))

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate) #Use Adam to optimize
sum_loss = []

In [ ]:
# Train the Model
for epoch in range(num_epochs):
	l = 0
	for i, (images, labels) in enumerate(train_loader):
		images = Variable(images)
		labels = Variable(labels)

		# Forward + Backward + Optimize
		optimizer.zero_grad()
		outputs = cnn(images)
		loss = criterion(outputs, labels)
		loss.backward()
		optimizer.step()
		l += loss.data[0]
		if (i+1) % 1 == 0:
			print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f'
				   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))
	sum_loss.append(l)

In [ ]:
cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in train_loader:
	images = Variable(images)
	outputs = cnn(images)
	_, predicted = torch.max(outputs.data, 1)
	total += labels.size(0)
	correct += (predicted == labels).sum()

print('Training Accuracy of the model on the test images: %d %%' % (100 * correct / total))